In [20]:
import pandas as pd, statsmodels.api as sm, matplotlib.pyplot as plt, numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from matplotlib import rc
from sklearn.model_selection import train_test_split, TimeSeriesSplit

from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [2]:
data_orig = pd.read_csv('parking-citations.csv')
data_orig.columns = data_orig.columns.str.replace(' ','')

data = data_orig[data_orig.IssueDate.notnull()]
data = data[['Ticketnumber', 'IssueDate']]
data.loc[:,'IssueDate'] = pd.to_datetime(data.loc[:,'IssueDate']).dt.to_period(freq='M')
data_grp = data.groupby(by='IssueDate').count()
data_grp.index = data_grp.index.to_timestamp()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [60]:
data_orig

,Ticketnumber,IssueDate,Issuetime,MeterId,MarkedTime,RPStatePlate,PlateExpiryDate,VIN,Make,BodyStyle,Color,Location,Route,Agency,Violationcode,ViolationDescription,Fineamount,Latitude,Longitude
0,1103341116,2015-12-21T00:00:00,1251.0,NaN,NaN,CA,200304.0,NaN,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,9.999900e+04,9.999900e+04
1,1103700150,2015-12-21T00:00:00,1435.0,NaN,NaN,CA,201512.0,NaN,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,9.999900e+04,9.999900e+04
2,1104803000,2015-12-21T00:00:00,2055.0,NaN,NaN,CA,201503.0,NaN,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6.439998e+06,1.802686e+06
3,1104820732,2015-12-26T00:00:00,1515.0,NaN,NaN,CA,NaN,NaN,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6.440041e+06,1.802686e+06
4,1105461453,2015-09-15T00:00:00,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,9.999900e+04,9.999900e+04
5,1106226590,2015-09-15T00:00:00,19.0,NaN,NaN,CA,201507.0,NaN,CHEV,VN,GY,SAN PEDRO S/O BOYD,1A35W,1.0,4000A1,NO EVIDENCE OF REG,50.0,9.999900e+04,9.999900e+04
6,1106500452,2015-12-17T00:00:00,1710.0,NaN,NaN,CA,201605.0,NaN,MAZD,PA,BL,SUNSET/ALVARADO,00217,1.0,8070,PARK IN GRID LOCK ZN,163.0,9.999900e+04,9.999900e+04
7,1106500463,2015-12-17T00:00:00,1710.0,NaN,NaN,CA,201602.0,NaN,TOYO,PA,BK,SUNSET/ALVARADO,00217,1.0,8070,PARK IN GRID LOCK ZN,163.0,9.999900e+04,9.999900e+04
8,1106506402,2015-12-22T00:00:00,945.0,NaN,NaN,CA,201605.0,NaN,CHEV,PA,BR,721 S WESTLAKE,2A75,1.0,8069AA,NO STOP/STAND AM,93.0,9.999900e+04,9.999900e+04
9,1106506413,2015-12-22T00:00:00,1100.0,NaN,NaN,CA,201701.0,NaN,NISS,PA,SI,1159 HUNTLEY DR,2A75,1.0,8069AA,NO STOP/STAND AM,93.0,9.999900e+04,9.999900e+04


In [41]:
data_orig_dropped = data_orig[['Agency','Fineamount']].dropna()

In [55]:
X = data_orig[['Violationcode']]
Y = data_orig[['Fineamount']]

In [56]:
X = pd.get_dummies(data=X, drop_first=True)

MemoryError: 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .20, random_state = 40)

In [ ]:
model = linear_model.LinearRegression()

In [46]:
model.fit(X_train, Y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').